<a href="https://colab.research.google.com/github/SveinungOverland/ZeroGO/blob/master/NN/DCNN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Neural Net for ZeroGO V1

In [74]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.__version__

'2.0.0'

## Layers and heads

### Residual layer

In [0]:
def add_residual_layer(x):
  res = layers.Conv2D(256, kernel_size=(3,3), padding='same')(x)
  res = layers.BatchNormalization(axis=1)(res)
  res = layers.ReLU()(res)
  res = layers.Conv2D(256, kernel_size=(3,3), padding='same')(res)
  res = layers.BatchNormalization(axis=1)(res)
  res = layers.ReLU()(res)
  res = layers.add([res, x])
  return layers.ReLU()(res)

### Create trunk

In [0]:
def create_trunk(shape, nr_residual_layers, filters):
  inputs = keras.Input(shape=shape)
  x = layers.Conv2D(filters, kernel_size=(3,3), padding='same')(inputs)
  x = layers.BatchNormalization(axis=1)(x)
  x = layers.ReLU()(x)
  for i in range(nr_residual_layers):
    x = add_residual_layer(x)
  return keras.Model(inputs, x)

### Create value head

In [0]:
def create_value_head(shape, filters):
    return keras.Sequential([
        layers.Conv2D(1, kernel_size=(1,1), padding='same'),
        layers.BatchNormalization(axis=1),
        layers.ReLU(),
        layers.Dense(filters),
        layers.ReLU(),
        layers.Dense(1, activation="tanh")
    ])

### Create policy head

In [0]:
def create_policy_head(shape, filters):
  return keras.Sequential([
      layers.Conv2D(2, kernel_size=(1,1), padding='same'),
      layers.BatchNormalization(axis=1),
      layers.ReLU(),
      layers.Dense(shape[0] * shape[1] + 1)
  ])

## Model class 

In [0]:
class Model:
  def __init__(self, trunk, value_head, policy_head):
    self.loss_fn = 
    
    # Can be choosen by mode
    self.trunk = trunk # Is a model
    self.value_head = value_head
    self.policy_head = policy_head
  
    # Can also be choosen by mode
    self.value_path = keras.Sequential([self.trunk, self.value_head])
    self.policy_path = keras.Sequential([self.trunk, self.policy_head])
    
  class Mode:
    Trunk = 0
    Value = 1
    Policy = 2
    ValueHead = 3
    PolicyHead = 4
    
  def save(self, file_name: str):
    print("Can't save yet")
    pass
  
  def __retrieve_net(self, mode):
    if mode == Mode.Trunk:
      net = self.trunk
    elif mode == Mode.Value:
      net = self.value_path
    elif mode == Mode.Policy:
      net = self.policy_path
    elif mode == Mode.ValueHead:
      net = self.value_head
    elif mode == Mode.PolicyHead:
      net = self.policy_head
    else:
      print("No acceptible mode given!")
      raise Exception("No acceptible mode given!")
    return net
  
  def train(self, mode: Mode, X, y):
    net = self.__retrieve_net(mode)
    with tf.GradientTape() as tape:
      prediction = net(X)
      loss = self.loss_fn(prediction, y)
    gradients = tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))
  
  def predict(self, mode, X):
    net = self.__retrieve_net(mode)
    return net(X)
  
  
  @classmethod
  def load(cls, filename):
    print("Can't load yet")
    pass
  
  @classmethod
  def create(cls, shape=(5,5,7), nr_residual_layers=10, filters=256):
    trunk = create_trunk(shape, nr_residual_layers, filters)
    value_head = create_value_head(shape, filters)
    policy_head = create_policy_head(shape, filters)
    return cls(trunk, value_head, policy_head)

## Test

In [0]:
model = Model.create(shape=(5,5,7), nr_residual_layers=2)